In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
  return a + b

agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  tools=[
    StructuredTool.from_function(
      func=plus,
      name="Sum Calculator",
      description="Use this to perfom sums of two numbers. This tool take two arguments, both should be numbers."
  ),
])

prompt = "Cost of $123.42 + $12932.12 + $121.3 + $120.59 + $1940.29 + $59.11"